Ce script permet d'afficher tous les trains traversant la France ayant du retard (et uniquement du retard).

# Code

## Import des Modules

In [178]:
import pandas as pd
import requests
import json
import datetime

## Obtention des trains ayant eu des perturbations sur leur trajet

Mise en varaible du token de connexion à l'API SNCF V1

In [179]:
token = 'e7b7fedd-71d0-48c6-8cc7-749e22ba8e80'

Requête à l'API SNCF afin de récupérer la liste des trains avec des perturbations d'aujourd'hui

L'API ne permettant que de récupérer 1000 résultats à la fois, on réalise une boucle afin de tout récupérer

In [180]:
# On récupère la date du jour
date = datetime.date.today()
date = str(date)
date = date[:4] + date[5:7] + date[8:10]

In [181]:
i = 0
val = True
df = pd.DataFrame()
while val:
    link = 'https://api.sncf.com/v1/coverage/sncf/disruptions?since=' + date +'T000000&start_page=' + str(i) + '&count=1000&'
    req = requests.get(link,auth=(token, ''))
    doc = json.loads(req.text)
    df = pd.concat([df, pd.DataFrame(doc['disruptions'])], ignore_index=True)
    i += 1
    if len(doc['disruptions']) != 1000:
        val = False

## Traitement des retards

On  ne garde que les informations sur les perturbations et le jour de ces dernières

In [182]:
df_retard = pd.DataFrame(list(df['severity']))
df_retard['updated_at'] = df['updated_at']

On ne garde que les trains ayant eu du retard

In [183]:
df_retard = df_retard[df_retard['effect'] == 'SIGNIFICANT_DELAYS']
df_retard = df_retard[['effect','updated_at']]

On réalise une fonction de conversion d'heures afin de calculer le retard

In [184]:
def conv_heure(string):
    return int(string[0:2]) + int(string[2:4])/60

## Obtention d'inforamtions complémentaires sur ce retard

On ajoute des informations au retard, ligne par ligne :

- Les gares de départ et d'arrivée

- Le retard et sa cause

- Les heures d'arrivée prévues et réelles du train

- L'heure de départ prévue du train

In [185]:
gare_d = []
gare_a = []
cause = []
retard = []
heure_arr_prevue = []
heure_arr_reelle = []
heure_depart = []

for i in df_retard.index:
    df_mod = pd.DataFrame(list(df['impacted_objects'][i]))
    df_mod = pd.DataFrame(list(df_mod['impacted_stops'])[0])
    df_mod = df_mod[(df_mod['amended_arrival_time'].notnull())]
    df_mod = df_mod[(df_mod['base_arrival_time'].notnull())]
    
    # On calcule le retard et on ajoute une exception si il y a un changement de jour entre les heures d'arrivée prévues et réelles
    retard_val = conv_heure(df_mod.iloc[-1]['amended_arrival_time']) - conv_heure(df_mod.iloc[-1]['base_arrival_time'])

    if retard_val<0:
        retard_val = int(round((retard_val+24)*60,1))
    else:
        retard_val = int(round(retard_val*60,1))
    
    df_mod['gare'] = pd.DataFrame(list(df_mod['stop_point']))['name']

    # On renvoie "Retard non expliqué" si la SNCF ne fournit pas plus d'informations sur le retard
    cause_ret = df_mod.iloc[-1]['cause']
    if cause_ret == "":
        cause_ret = "Retard non expliqué"

    # On ajoute les informations à des listes
    gare_d.append(df_mod.iloc[0]['gare'])
    gare_a.append(df_mod.iloc[-1]['gare'])
    cause.append(cause_ret)
    retard.append(retard_val)
    heure_arr_prevue.append(df_mod.iloc[-1]['base_arrival_time'])
    heure_arr_reelle.append(df_mod.iloc[-1]['amended_arrival_time'])
    heure_depart.append(df_mod.iloc[0]['base_departure_time'])


Création d'une fonction afin d'obtenir le jour du départ

In [186]:
def get_day(string):
    string = string[:8]
    return string[6:8]+'-'+string[4:6]+'-'+string[0:4]

On ajoute les informations au dataframe

In [187]:
df_retard = df_retard.drop(['effect'], axis=1)

df_retard['Gare (départ)'] = gare_d
df_retard['Gare (arrivée)'] = gare_a
df_retard['updated_at'] = df_retard['updated_at'].apply(get_day)
df_retard["Arrivée (réelle)"] = heure_arr_reelle
df_retard["Arrivée (prévue)"] = heure_arr_prevue
df_retard["Départ (prévu)"] = heure_depart
df_retard['Retard (min)'] = retard
df_retard['Cause'] = cause

df_retard.rename(columns = {'updated_at':'Jour'}, inplace = True)


On passe les heures au format heure de `datetime`

In [188]:
def str_tps(str):
    return datetime.datetime.strptime(str, '%H%M%S').time()

df_retard["Arrivée (réelle)"] = df_retard["Arrivée (réelle)"].apply(str_tps)
df_retard["Arrivée (prévue)"] = df_retard["Arrivée (prévue)"].apply(str_tps)
df_retard["Départ (prévu)"] = df_retard["Départ (prévu)"].apply(str_tps)

On affiche le dataframe final

In [189]:
df_retard

,Jour,Gare (départ),Gare (arrivée),Arrivée (réelle),Arrivée (prévue),Départ (prévu),Retard (min),Cause
4,04-03-2023,Saint-Étienne Châteaucreux,Montbrison,09:17:30,09:12:30,08:33:00,5,Difficultés lors de la préparation du train
8,04-03-2023,Lyon Part Dieu,Aix-les-Bains - Le Revard,18:23:00,18:18:00,17:08:00,5,Saturation des voies en gare
41,04-03-2023,Marseille - Saint-Charles,Hyères,13:01:00,12:56:00,11:32:00,5,Retard non expliqué
50,04-03-2023,Paris - Gare de Lyon - Hall 1 & 2,Évian-les-Bains,11:35:30,11:15:30,06:49:00,20,Incident de circulation
53,04-03-2023,Orléans,Tours,19:03:00,18:58:00,17:38:00,5,Prise en charge de clients en correspondance
...,...,...,...,...,...,...,...,...
3925,04-03-2023,Stuttgart Hbf,Paris Est,23:16:00,22:06:00,18:52:00,70,Défaillance de matériel
3926,04-03-2023,Montréjeau - Gourdan-Polignan,Toulouse Matabiau,21:39:30,21:29:30,20:13:00,10,Défaillance de matériel
3927,04-03-2023,Strasbourg,Paris Est,23:19:30,22:34:30,20:01:00,45,Présence d'un bagage abandonné
3928,04-03-2023,Aix-les-Bains - Le Revard,Lyon Part Dieu,20:57:00,20:52:00,19:41:00,5,Régulation du trafic


On exporte le dataframe en fichier _CSV_

In [190]:
df_retard.to_csv('Delay.csv', sep=',', index=False, header=True)